In [2]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import scanpy as sc

In [3]:
import scanpy as sc
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import anndata2ri
import logging
from scipy.sparse import issparse

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

In [4]:
adata = sc.read('/orcd/data/lhtsai/001/om2/mabdel03/files/ACE_Analysis/Analysis/Tsai/Processing/ACE/Final_Pipeline/Batch_Correction/batch_corrected.h5ad')

/orcd/data/lhtsai/001/om2/mabdel03/conda_envs/single_cell_BP/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


# Normalization

In [5]:
%load_ext rpy2.ipython

In [6]:
adata.layers["counts"] = adata.X.copy()

## Shifted Logarithm Normalization

In [7]:
#Scale and log transform the counts
scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
# log1p transform
adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

## Scran Normalization

In [8]:
from scipy.sparse import csr_matrix, issparse

In [9]:
%%R
library(scran)
library(BiocParallel)

In [ ]:
import scanpy as sc
import numpy as np
from scipy.sparse import issparse, csr_matrix
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Activate the pandas to R DataFrame conversion
pandas2ri.activate()

# Import the required R packages
scran = importr('scran')
SingleCellExperiment = importr('SingleCellExperiment')
BiocParallel = importr('BiocParallel')

# Define the necessary R functions using 'get'
sizeFactors = ro.r('get("sizeFactors", asNamespace("scran"))')
computeSumFactors = ro.r('get("computeSumFactors", asNamespace("scran"))')
MulticoreParam = ro.r('BiocParallel::MulticoreParam')

# Preliminary clustering for differentiated normalization
adata_pp = adata.copy()
sc.pp.normalize_total(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="groups")

data_mat = adata_pp.X.T
# Convert to a dense format if necessary
if issparse(data_mat):
    data_mat = data_mat.todense()

ro.globalenv["data_mat"] = ro.r.matrix(data_mat, nrow=data_mat.shape[0], ncol=data_mat.shape[1])
ro.globalenv["input_groups"] = ro.FactorVector(adata_pp.obs["groups"])

del adata_pp

# Compute size factors in R
ro.r('size_factors <- sizeFactors(computeSumFactors(SingleCellExperiment(list(counts=data_mat)), clusters = input_groups, min.mean = 0.1, BPPARAM = MulticoreParam()))')
size_factors = np.array(ro.globalenv["size_factors"])
adata.obs["size_factors"] = size_factors

scran = adata.X / adata.obs["size_factors"].values[:, None]
obj.layers["scran_normalization"] = csr_matrix(sc.pp.log1p(scran))

/orcd/data/lhtsai/001/om2/mabdel03/conda_envs/single_cell_BP/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


## Analytic Pearson Residuals

In [ ]:
import scanpy as sc
import numpy as np
from scipy.sparse import csr_matrix

# Define a small epsilon value to avoid division by zero
epsilon = 1e-10

# Apply Pearson residual normalization
analytic_pearson = sc.experimental.pp.normalize_pearson_residuals(adata, inplace=False)

# Handle potential NaN or infinite values by replacing them with zeros
residuals = analytic_pearson["X"]
residuals = np.nan_to_num(residuals, nan=0.0, posinf=0.0, neginf=0.0)

# Add epsilon to avoid division by zero
residuals = residuals / (np.sqrt(np.maximum(residuals, epsilon) + np.maximum(residuals**2 / epsilon, epsilon)))

adata.layers["analytic_pearson_residuals"] = csr_matrix(residuals)


# Variable Features

In [ ]:
import scanpy as sc
import os
import anndata2ri
import logging
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import anndata as adata

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(scry)

In [ ]:
# Ensure the AnnData object uses compatible data types
adata.X = adata.X.astype(float)
obj_name = f'adata'
ro.globalenv[obj_name] = adata

# Use R magic command to run devianceFeatureSelection
%R -i obj_name sce <- devianceFeatureSelection(get(obj_name), assay="X") #obj_name is the input object to R
%R -o binomial_deviance binomial_deviance <- rowData(sce)$binomial_deviance #binomial_deviance is output to python

# Convert binomial_deviance to numpy array
binomial_deviance = np.array(binomial_deviance)

idx = binomial_deviance.argsort()[-4000:] 
mask = np.zeros(obj.var_names.shape, dtype=bool)
mask[idx] = True

obj.var["highly_deviant"] = mask
obj.var["binomial_deviance"] = binomial_deviance

sc.pp.highly_variable_genes(obj, layer="scran_normalization")

# Scale Data 

In [ ]:
adata.layers['scaled_data'] = sc.pp.scale(data=adata.layers['log1p_norm'], copy=True)

# Dimensionality Reduction

## PCA

In [ ]:
adata.X = adata.layers["scaled_data"]
adata.var["highly_variable"] = adata.var["highly_deviant"]
sc.pp.pca(adata, svd_solver="arpack", use_highly_variable=True)
sc.pl.pca_scatter(adata, color="total_counts")

## UMAP

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.pl.umap(adata, color="total_counts")

## QC Inspection

In [ ]:
sc.pl.umap(
        obj,
        color=["total_counts", "pct_counts_mt", "scDblFinder_score", "scDblFinder_class"],
    )

# Clustering